In [ ]:
# installing required libraries
!pip install geopandas
!add-apt-repository ppa:ubuntugis/ppa -y
!apt-get update
!apt-get install python-numpy gdal-bin libgdal-dev python3-rtree
!pip install rasterio

     |████████████████████████████████| 994 kB 4.9 MB/s 
     |████████████████████████████████| 15.4 MB 36 kB/s 
     |████████████████████████████████| 6.6 MB 38.5 MB/s 
Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim

%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import skimage
import os


%cd /content/gdrive/MyDrive/SpaceNet
import libs.solaris as sol # modified solaris library


from glob import glob
from utils.dist import *
from data.dataset import *

Mounted at /content/gdrive
/content/gdrive/MyDrive/SpaceNet


In [ ]:
# load data and define transform
import torchvision
import torchvision.transforms as transforms

class Resize(object):
  def __init__(self, size):
    self.size = size
  def __call__(self, img):
    return skimage.transform.resize(img, self.size)

transform = transforms.Compose([transforms.ToTensor()])
target_transform = transforms.Compose([Resize((162,162)),
                                       transforms.ToTensor()])
dataset = Spacenet(transform=transform, target_transform = target_transform)

train, validation = torch.utils.data.random_split(dataset, [int(len(dataset)*0.8), len(dataset) - int(len(dataset)*0.8)])
train_loader = torch.utils.data.DataLoader(train, batch_size = 5, shuffle=True, num_workers=2)
validation_loader = torch.utils.data.DataLoader(validation, batch_size=5, shuffle=True, num_workers=2)

In [ ]:
# baseline model
from models.baseline import *
device = 'cuda'

net = FCnet(8, 1)
net = net.to(device)
#cudnn.benchmark = True

file_name = 'FCNN_khartoum.pt'
trained_dir = '/content/gdrive/MyDrive/SpaceNet/trained'

criterion = nn.MSELoss(reduction='sum') # L2 Loss, matrix에서 loss계산?
optimizer = optim.Adam(net.parameters(), lr=0.001, weight_decay = 0.1)

In [ ]:
# train and evaluate
from utils.eval import *
import math
val_loss = []

def train(epoch):
  print('\n Current train epoch : ', epoch)
  net.train()
  for batch_idx, (imgs, targets) in enumerate(train_loader):
    imgs, targets = imgs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(imgs)

    loss = criterion(outputs, targets)
  
    loss.backward()
    if math.isnan(loss.item()):
      raise KeyboardInterrupt('nan detected')
    print('loss: ', loss.item())
    optimizer.step()

def test(epoch):
  print('\n current validation epoch: ', epoch)
  net.eval()
  total_loss = 0
  for batch_idx, (imgs, targets) in enumerate(validation_loader):
    imgs, targets = imgs.to(device), targets.to(device)
    optimizer.zero_grad()
    outputs = net(imgs)

    loss = criterion(outputs, targets)
    total_loss += loss.item()/10 # divide by batch size 

  state = {
      'net': net.state_dict(),
      'epoch': epoch,
      'optimizer': optimizer.state_dict()
  }
  torch.save(state, os.path.join(trained_dir, file_name))
  val_loss.append(total_loss)
  print('\n Current loss : ', total_loss) # get segmentation metrics 


In [ ]:
for epoch in range(1, 60):
  train(epoch)
  test(epoch)


 Current train epoch :  24


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


loss:  9726.2431640625
loss:  11210.3818359375
loss:  11450.6240234375
loss:  21771.947265625
loss:  9271.8486328125
loss:  8894.822265625
loss:  15242.826171875
loss:  25733.46484375
loss:  17846.58203125
loss:  9911.0869140625
loss:  10389.5380859375
loss:  13150.0048828125
loss:  13634.98046875
loss:  7533.38720703125
loss:  8246.90234375
loss:  8541.8779296875
loss:  9677.763671875
loss:  9901.98828125
loss:  8685.041015625
loss:  17621.396484375
loss:  9656.9365234375
loss:  6575.07080078125
loss:  12029.3916015625
loss:  9709.5048828125
loss:  8844.44921875
loss:  8406.5791015625
loss:  14113.466796875
loss:  21155.521484375
loss:  9198.123046875
loss:  9894.494140625
loss:  9648.646484375
loss:  10133.0400390625
loss:  7031.6220703125
loss:  6479.216796875
loss:  13917.1357421875
loss:  7818.27880859375
loss:  8405.296875
loss:  9136.1484375
loss:  6782.4658203125
loss:  23180.0078125
loss:  7998.62841796875
loss:  18904.818359375
loss:  8878.04296875
loss:  7094.6279296875
loss